# Debug COEFF columns in specprod-db

In [1]:
import os
import numpy as np
from astropy.table import Table
from desiutil.log import get_logger
from desispec.io import findfile
import specprodDB.load as db
from sqlalchemy.sql.expression import func

In [2]:
specprod = os.environ['SPECPROD'] = 'fuji'
# specprod = os.environ['SPECPROD'] = 'guadalupe'
# specprod = os.environ['SPECPROD'] = 'iron'
# specprod = os.environ['SPECPROD'] = 'loa'
sample_size = 100  # Number of random samples to retrieve from the database.

In [3]:
db.log = get_logger()
postgresql = db.setup_db(schema=specprod, hostname='specprod-db.desi.lbl.gov', username='desi')

In [7]:
db.dbSession.rollback()

In [8]:
# Query on individual columns to allow this to run on fuji.
random_zpix_query = db.dbSession.query(db.Zpix.targetid, db.Zpix.survey, db.Zpix.program, db.Zpix.healpix, db.Zpix.coeff_0, db.Zpix.coeff_1, db.Zpix.coeff_2, db.Zpix.coeff_3, db.Zpix.coeff_4, db.Zpix.coeff_5, db.Zpix.coeff_6, db.Zpix.coeff_7, db.Zpix.coeff_8, db.Zpix.coeff_9).order_by(func.random()).limit(sample_size).all()

In [10]:
random_zpix = dict()
for row in random_zpix_query:
    # print(row)
    filename, exists = findfile('redrock', survey=row[1], faprogram=row[2], healpix=row[3], readonly=True, return_exists=True)
    # return_exists is acting weird for /dvs_ro and loa.
    if not exists:
        exists = os.path.exists(filename)
    if exists:
        key = (row[0], row[1], row[2])  # targetid, survey, program
        random_zpix[key] = {'row': row, 'filename': filename}
        random_zpix[key]['db_coeff'] = np.array(row[4:])
    else:
        print(f"WARNING: {filename} does not exist, skipping!")

In [12]:
random_ztile_subquery = db.dbSession.query(db.Fiberassign).order_by(func.random()).limit(sample_size).subquery()
random_ztile_query = db.dbSession.query(db.Ztile.targetid, db.Ztile.tileid, db.Ztile.spgrpval, random_ztile_subquery.c.petal_loc, db.Ztile.coeff_0, db.Ztile.coeff_1, db.Ztile.coeff_2, db.Ztile.coeff_3, db.Ztile.coeff_4, db.Ztile.coeff_5, db.Ztile.coeff_6, db.Ztile.coeff_7, db.Ztile.coeff_8, db.Ztile.coeff_9).join(random_ztile_subquery, (db.Ztile.targetid==random_ztile_subquery.c.targetid) & (db.Ztile.tileid==random_ztile_subquery.c.tileid)).all()

In [13]:
random_ztile = dict()
for row in random_ztile_query:
    # print(row)
    filename, exists = findfile('redrock', groupname='cumulative', tile=row[1], night=row[2], spectrograph=row[3], readonly=True, return_exists=True)
    if not exists:
        exists = os.path.exists(filename)
    if exists:
        key = (row[0], row[1])  # targetid, tileid
        random_ztile[key] = {'row': row, 'filename': filename}
        random_ztile[key]['db_coeff'] = np.array(row[4:])
    else:
        print(f"WARNING: {filename} does not exist, skipping!")

In [14]:
random_photometry_query = db.dbSession.query(db.Photometry).order_by(func.random()).limit(sample_size).all()

In [15]:
random_photometry = dict()
for row in random_photometry_query:
    if row.release == 9011:
        ns = 'north'
    else:
        ns = 'south'
    if row.brickname:
        ra = row.brickname[0:3]
        filename = f"/dvs_ro/cfs/cdirs/cosmo/data/legacysurvey/dr9/{ns}/tractor/{ra}/tractor-{row.brickname}.fits"
        exists = os.path.exists(filename)
        if exists:
            key = row.targetid
            random_photometry[key] = {'row': row, 'filename': filename}
            random_photometry[key]['db_dchisq'] = np.array([row.dchisq_psf, row.dchisq_rex, row.dchisq_dev, row.dchisq_exp, row.dchisq_ser])
        else:
            print(row)
            print(f"WARNING: {filename} does not exist, skipping!")
    else:
        print(f"INFO: Likely secondary target, {row}, skipping.")

INFO: Likely secondary target, Photometry(targetid=238961184211186), skipping.
INFO: Likely secondary target, Photometry(targetid=2305843032341210396), skipping.
INFO: Likely secondary target, Photometry(targetid=243323029684394), skipping.


In [21]:
randoms = {'zpix': random_zpix, 'ztile': random_ztile, 'photometry': random_photometry}
hdus = {'zpix': 'REDSHIFTS', 'ztile': 'REDSHIFTS', 'photometry': 1}
for table in randoms:
    for key in randoms[table]:
        data = Table.read(randoms[table][key]['filename'], format='fits', hdu=hdus[table])
        if table == 'photometry':
            w = np.where(data['objid'] == randoms[table][key]['row'].brick_objid)[0]
        else:
            w = np.where(data['TARGETID'] == randoms[table][key]['row'][0])[0]
        if len(w) == 0:
            print(f"ERROR: object not found in {randoms[table][key]['filename']}!")
        elif len(w) > 1:
            print(f"ERROR: multiple objects found in {randoms[table][key]['filename']}!")
        else:
            if table == 'photometry':
                assert np.allclose(data[w]['dchisq'].data[0], randoms[table][key]['db_dchisq'])
            else:
                if specprod == 'loa':
                    assert np.allclose(data[w]['COEFF'].data[0], randoms[table][key]['db_coeff'])
                else:
                    try:
                        assert not np.allclose(data[w]['COEFF'].data[0], randoms[table][key]['db_coeff'])
                    except AssertionError:
                        print(specprod, table, key, data[w]['COEFF'].data[0], randoms[table][key]['db_coeff'])

fuji zpix (39633493575076941, 'sv1', 'dark') [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
fuji zpix (39632936286292426, 'sv1', 'dark') [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
